In [1]:
import requests
import json
import csv
import os
from datetime import datetime
import pandas as pd
import logging
import boto3
from botocore.exceptions import ClientError

# Bitmex
https://www.bitmex.com/api/explorer/#/

use XBTUSD as currency

In [19]:
r = requests.get('https://www.bitmex.com/api/v1/stats/history')
if r.status_code != 200:
    print(r.status_code)

data = r.json()
filtered_data = [data[i] for i in range(len(data)) if (data[i]['rootSymbol']=='XBT') | (data[i]['rootSymbol']=='ETH')]
filtered_data

[{'date': '2014-11-22T00:00:00.000Z',
  'rootSymbol': 'XBT',
  'currency': 'XBt',
  'volume': 2,
  'turnover': 0},
 {'date': '2014-11-23T00:00:00.000Z',
  'rootSymbol': 'XBT',
  'currency': 'XBt',
  'volume': 2,
  'turnover': 0},
 {'date': '2014-11-24T00:00:00.000Z',
  'rootSymbol': 'XBT',
  'currency': 'XBt',
  'volume': 30315,
  'turnover': 0},
 {'date': '2014-11-25T00:00:00.000Z',
  'rootSymbol': 'XBT',
  'currency': 'XBt',
  'volume': 49220,
  'turnover': 0},
 {'date': '2014-11-26T00:00:00.000Z',
  'rootSymbol': 'XBT',
  'currency': 'XBt',
  'volume': 29212,
  'turnover': 0},
 {'date': '2014-11-27T00:00:00.000Z',
  'rootSymbol': 'XBT',
  'currency': 'XBt',
  'volume': 282,
  'turnover': 0},
 {'date': '2014-11-28T00:00:00.000Z',
  'rootSymbol': 'XBT',
  'currency': 'XBt',
  'volume': 5927,
  'turnover': 0},
 {'date': '2014-11-29T00:00:00.000Z',
  'rootSymbol': 'XBT',
  'currency': 'XBt',
  'volume': 3142,
  'turnover': 39269000},
 {'date': '2014-11-30T00:00:00.000Z',
  'rootSymbol':

## 1. Download historical data

In [7]:
now = datetime.now().isoformat()
bitmex_params = [('funding',{'symbol':'XBTUSD','count':'100','reverse':'false','start':'0','endTime':now}),    # funding_rate
                 #('stats/history',{}),    # volume
                 #('stats',{}),    # openInterest, openValue "rootSymbol" -- call "XBT" and "ETH"
                ]

# Initialise historical data. This will not overwrite files of the same name in the target folder so be sure to delete them first if they exist.
def get_historical_bitmex(bitmex_params):
    sample = 'sample'
    for item in bitmex_params:
        endpoint, params_dict = item
        filepath = 'data/Bitmex/' + endpoint.replace('/', '_') + '.csv'
        
        if os.path.isfile(filepath):
            print('Please delete or rename previous file of same name.')
            raise
        
        while sample!='':
            api_call_url = 'https://www.bitmex.com/api/v1/' + endpoint + '?_format=csv'
            # Check if any parameters are applied
            if bool(params_dict):
                for param in params_dict:
                    api_call_url += '&' + param + '=' + params_dict[param] 
            r = requests.get(api_call_url)
            if r.status_code != 200:
                print(r.status_code)
                return
            csv_list = r.text.split('\n')
            
            # Create a new file only if one does not exist, and append the headers
            if not os.path.isfile(filepath):
                with open(filepath, 'w') as csv_file:
                    for row in csv_list[0]:
                        csv_file.write(row)
            
            with open(filepath, 'a') as csv_file:
                # Append the data, but skip the headers
                for row in csv_list[1:]:
                    csv_file.write('\n'+row)
            
            if bool(params_dict):
                params_dict['start'] = str(int(params_dict['start']) + int(params_dict['count']))
                sample = r.text[:5]
            else:
                sample = ''

In [8]:
get_historical_bitmex(bitmex_params)

## 2. Update data

In [46]:
# Ref: https://www.bitmex.com/api/explorer/
# Parameters for each API call. All variables must be in string format
bitmex_params = [('funding',{'symbol':'XBTUSD','count':'100','reverse':'false','start':'0'}),
                 #('liquidation',{}),
                 #('stats/history',{}),
                ]

def update_bitmex(bitmex_params):
    # Update data from timestamp of last entry in file
    for item in bitmex_params:
        endpoint, params_dict = item
        filepath = 'data/Bitmex/' + endpoint + '.csv'
        api_call_url = 'https://www.bitmex.com/api/v1/' + endpoint + '?_format=csv'
        
        # Check if any parameters are applied
        if bool(params_dict):
            # Check for timestamp of last entry
            if endpoint == 'funding':
                timestamp_df = pd.read_csv(filepath, usecols=['timestamp'])
            elif endpoint == 'stats/history':
                timestamp_df = pd.read_csv(filepath, usecols=['date'])
            elif endpoint == 'stats':
                timestamp_df = pd.read_csv(filepath, usecols=['timestamp'])

            params_dict['startTime'] = timestamp_df.values[-1][0]
            params_dict['start'] = '1'
            
            for param in params_dict:
                api_call_url += '&' + param + '=' + params_dict[param] 

        r = requests.get(api_call_url)
        if r.status_code != 200:
            print(r.status_code)
            break
        
        csv_list = r.text.split('\n')
        with open(filepath, 'a') as csv_file:
            # Append the data, but skip the headers
            for row in csv_list[1:]:
                csv_file.write(row + '\n')
                
update_bitmex(bitmex_params)

### Stats

In [18]:
r = requests.get('https://www.bitmex.com/api/v1/stats')
if r.status_code != 200:
    print(r.status_code)

data = r.json()
filtered_data = [data[i] for i in range(len(data)) if (data[i]['rootSymbol']=='XBT') | (data[i]['rootSymbol']=='ETH')]

stats_df = pd.DataFrame(filtered_data)
stats_df['Time'] = datetime.now()
stats_df.to_csv('data/Bitmex/stats.csv', mode='a', header=False, index=False)

### Stats history

In [20]:
r = requests.get('https://www.bitmex.com/api/v1/stats/history')
if r.status_code != 200:
    print(r.status_code)

data = r.json()
filtered_data = [data[i] for i in range(len(data)) if (data[i]['rootSymbol']=='XBT') | (data[i]['rootSymbol']=='ETH')]

stats_hist_df = pd.DataFrame(filtered_data)
stats_hist_df.to_csv('data/Bitmex/stats_history.csv', mode='w', index=False)

# Coinmetrics
https://docs.coinmetrics.io/api/v2/

## a) Get historical data

In [146]:
# Get full data
metrics = 'AdrActCnt,BlkCnt,BlkSizeByte,BlkSizeMeanByte,CapMVRVCur,CapMrktCurUSD,CapRealUSD,DiffMean,FeeMeanNtv,FeeMeanUSD,FeeMedNtv,FeeMedUSD,FeeTotNtv,FeeTotUSD,IssContNtv,IssContPctAnn,IssContUSD,IssTotNtv,IssTotUSD,NVTAdj,NVTAdj90,PriceBTC,PriceUSD,ROI1yr,ROI30d,SplyCur,TxCnt,TxTfrCnt,TxTfrValAdjNtv,TxTfrValAdjUSD,TxTfrValMeanNtv,TxTfrValMeanUSD,TxTfrValMedNtv,TxTfrValMedUSD,TxTfrValNtv,TxTfrValUSD,VtyDayRet180d,VtyDayRet30d,VtyDayRet60d'
r = requests.get('https://community-api.coinmetrics.io/v2/assets/btc/metricdata.csv?metrics='+ metrics)
if r.status_code != 200:
    print(r.status_code)
    raise

In [147]:
csv_list = r.text.split('\n')
with open('data/Coinmetrics/Coinmetrics_btc.csv', 'w') as csv_file:
    for row in csv_list:
        csv_file.write(row)

## b) Update daily

In [3]:
# Get timestamp of last entry
filepath = 'data/Coinmetrics/Coinmetrics_btc.csv'
timestamp_df = pd.read_csv(filepath, usecols=['time'])
timestamp_df = timestamp_df.astype('datetime64')
start_time = str(timestamp_df.values[-1][0]+10000000)
start = '&start=' + str(start_time)[:23] + 'Z'

metrics = 'AdrActCnt,BlkCnt,BlkSizeByte,BlkSizeMeanByte,CapMVRVCur,CapMrktCurUSD,CapRealUSD,DiffMean,FeeMeanNtv,FeeMeanUSD,FeeMedNtv,FeeMedUSD,FeeTotNtv,FeeTotUSD,IssContNtv,IssContPctAnn,IssContUSD,IssTotNtv,IssTotUSD,NVTAdj,NVTAdj90,PriceBTC,PriceUSD,ROI1yr,ROI30d,SplyCur,TxCnt,TxTfrCnt,TxTfrValAdjNtv,TxTfrValAdjUSD,TxTfrValMeanNtv,TxTfrValMeanUSD,TxTfrValMedNtv,TxTfrValMedUSD,TxTfrValNtv,TxTfrValUSD,VtyDayRet180d,VtyDayRet30d,VtyDayRet60d' #,TxTfr
r = requests.get('https://community-api.coinmetrics.io/v2/assets/btc/metricdata.csv?metrics='+ metrics + start)
if r.status_code != 200:
    print(r.status_code)
    raise
csv_list = r.text.split('\n')
with open(filepath, 'a') as csv_file:
    for row in csv_list[1:]:
        csv_file.write(row)

# Quandl
https://www.quandl.com/tools/python

In [4]:
import quandl

## Get historical data

In [57]:
for code in ["BCHAIN/MIREV","BCHAIN/DIFF","BCHAIN/HRATE","BCHAIN/TOTBC","BCHAIN/TOUTV"]:
    data_df = quandl.get(code)
    data_df.to_csv('data/Quandl/'+ code.replace('/','_')+'.csv')

Error : Connection is already closed.
error from callback <bound method BitMEXWebsocket.__on_error of <bitmex_websocket.BitMEXWebsocket object at 0x000002183B20BF98>>: Connection is already closed.
Error : Connection is already closed.
error from callback <bound method BitMEXWebsocket.__on_error of <bitmex_websocket.BitMEXWebsocket object at 0x000002183C03DF28>>: Connection is already closed.


## Update data

In [5]:
for code in ["BCHAIN/MIREV","BCHAIN/DIFF","BCHAIN/HRATE","BCHAIN/TOTBC","BCHAIN/TOUTV"]:
    old_data_df = pd.read_csv('data/Quandl/'+ code.replace('/','_') + '.csv', usecols=['Date'])
    final_timestamp = old_data_df.astype('datetime64').values[-1] + pd.Timedelta(1, unit='d')
    start = str(final_timestamp[0])[:10]
    data_df = quandl.get(code, start_date=start)
    # Append    
    data_df.to_csv('data/Quandl/'+ code.replace('/','_') + '.csv', mode='a', header=False)

LimitExceededError: (Status 429) (Quandl Error QELx01) You have exceeded the anonymous user limit of 50 calls per day. To make more calls today, please register for a free Quandl account and then include your API key with your requests.

# Glassnode

## Get historical

In [12]:
variables = ['sopr', 'sopr_adjusted', 'cdd', 'average_dormancy', 'average_dormancy_supply_adjusted']
for var in variables:
    r = requests.get('https://api.glassnode.com/v1/metrics/indicators/'+ var + '?a=btc&api_key=022517a3-d79c-4510-9063-8d83780bb927')
    r.raise_for_status()
    json_file = r.json()
    glassnode_df = pd.DataFrame(json_file)
    glassnode_df['Time'] = pd.to_datetime(glassnode_df['t'], unit='s')
    glassnode_df.to_csv('data/Glassnode/' + var + '.csv', index=False)

## Update Data

In [32]:
variables = ['sopr', 'sopr_adjusted', 'cdd', 'average_dormancy', 'average_dormancy_supply_adjusted']
for var in variables:
    old_data_df = pd.read_csv('data/Glassnode/' + var + '.csv', usecols=['t'])
    final_timestamp = old_data_df.values[-1][0]
    # Filter from the time of 1 hour from last timestamp
    r = requests.get('https://api.glassnode.com/v1/metrics/indicators/'+ var + '?a=btc&s='+ str(final_timestamp+86400) + '&api_key=022517a3-d79c-4510-9063-8d83780bb927')
    r.raise_for_status()
    json_file = r.json()
    if json_file:
        new_timestamp = json_file[0]['t']
        glassnode_df = pd.DataFrame(json_file)
        glassnode_df['Time'] = pd.to_datetime(glassnode_df['t'], unit='s')
        glassnode_df.to_csv('data/Glassnode/' + var + '.csv', mode='a', header=False, index=False)

# Upload to S3

In [82]:
def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = file_name

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [83]:
# Let's use Amazon S3
s3 = boto3.resource('s3')
# Print out bucket names
for bucket in s3.buckets.all():
    print(bucket.name)

your-s3-bucket-name-deeloon


In [84]:
data = open('data.json', 'rb')
s3.Bucket('your-s3-bucket-name-deeloon').put_object(Key='data.json', Body=data)

s3.Object(bucket_name='your-s3-bucket-name-deeloon', key='data.json')